In [32]:
import pandas as pd
import numpy as np
from snappy import *
#from sage.all import *


In [33]:
#load data

#knotinfo = pd.read_excel('/Users/seohyeonlee/knotinfo/data/knotinfo_data_complete.xls')
linkinfo = pd.read_excel('/Users/seohyeonlee/knotinfo/data/linkinfo_data_complete.xls')


In [34]:
linkinfo_glossary = linkinfo.iloc[0, :].copy()
linkinfo_glossary['determinant']

linkinfo_glossary.shape


(89,)

In [35]:
linkinfo.shape
linkinfo.describe()
linkinfo.isna().any()
linkinfo.head()

#remove first row
linkinfo = linkinfo.iloc[1:, :].reset_index(drop=True)
linkinfo


,name,name_anon,diagram,diagram_anon,name_unoriented,name_unoriented_anon,category,category_anon,knot_atlas,knot_atlas_anon,...,determinant_anon,signature,signature_anon,nullity,nullity_anon,unlinking_number,unlinking_number_anon,Unnamed: 86,weak_splitting_number,weak_splitting_number_anon
0,L2a1{0},diagram_display.php?L2a1{0},linkL2a1{0}-50.png,diagram_display.php?L2a1{0},L2a1,NaN,2,NaN,L2a1,http://katlas.math.toronto.edu/wiki/L2a1,...,NaN,1,NaN,0,NaN,1,NaN,NaN,1,NaN
1,L2a1{1},diagram_display.php?L2a1{1},linkL2a1{1}-50.png,diagram_display.php?L2a1{1},L2a1,NaN,2,NaN,L2a1,http://katlas.math.toronto.edu/wiki/L2a1,...,NaN,-1,NaN,0,NaN,1,NaN,NaN,1,NaN
2,L4a1{0},diagram_display.php?L4a1{0},linkL4a1{0}-50.png,diagram_display.php?L4a1{0},L4a1,NaN,4,NaN,L4a1,http://katlas.math.toronto.edu/wiki/L4a1,...,NaN,1,NaN,0,NaN,2,NaN,NaN,2,NaN
3,L4a1{1},diagram_display.php?L4a1{1},linkL4a1{1}-50.png,diagram_display.php?L4a1{1},L4a1,NaN,4,NaN,L4a1,http://katlas.math.toronto.edu/wiki/L4a1,...,NaN,-3,NaN,0,NaN,2,NaN,NaN,2,NaN
4,L5a1{0},diagram_display.php?L5a1{0},linkL5a1{0}-50.png,diagram_display.php?L5a1{0},L5a1,NaN,5,NaN,L5a1,http://katlas.math.toronto.edu/wiki/L5a1,...,NaN,1,NaN,0,NaN,1,NaN,NaN,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8370,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8371,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8372,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8373,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
#4000+ knots missing name
linkinfo['name'].value_counts()

type(linkinfo['name'][8373])

noname = linkinfo.loc[linkinfo['name'].isna()]
noname.isna().sum()


name                          4187
name_anon                     4187
diagram                       4187
diagram_anon                  4187
name_unoriented               4187
                              ... 
unlinking_number              4187
unlinking_number_anon         4187
Unnamed: 86                   4187
weak_splitting_number         4187
weak_splitting_number_anon    4187
Length: 89, dtype: int64

In [37]:
#remove rows where all values are NaN. do the same with columns
linkinfo = linkinfo.dropna(axis=0, how='all')
linkinfo = linkinfo.dropna(axis=1, how='all')


In [38]:
#sage

#extract properties needed to determine if link is ribbon
alexander = linkinfo['multivariable_alexander']
jones = linkinfo['jones_polynomial']
linking_matrix = linkinfo['linking_matrix']
#check jones polynomial
#check if factorable by the jones polynomial of the unlink with same components: (q + q^{-1})^{n-1}
linkinfo['jones_polynomial']


0                                          -x^(-5)-x^(-1)
1                                                  -x-x^5
2                          -x^(-9)-x^(-5) + x^(-3)-x^(-1)
3                                     -x^3-x^7 + x^9-x^11
4                       x^(-7)-2/x^5 + x^(-3)-2/x + x-x^3
                              ...                        
4183    -x^(-5) + x^(-3)-3/x-2*x^3-x^5-2*x^9 + x^11-2*...
4184    -x^(-17) + x^(-15)-3/x^13-2/x^9-x^(-7)-2/x^3 +...
4185    -x^(-11) + x^(-9)-3/x^7-2/x^3-x^(-1)-2*x^3 + x...
4186    -x^(-11) + x^(-9)-3/x^7-2/x^3-x^(-1)-2*x^3 + x...
4187    -x^7 + x^9-3*x^11-2*x^15-x^17-2*x^21 + x^23-2*...
Name: jones_polynomial, Length: 4188, dtype: object

In [39]:
import  re

#process linking matrices
#'matrix_zero' is true if the linking matrix consists of only 0s, false if not, NA if data is missing

def parse_matrix(row):
	if not pd.isna(row):
		nums = re.findall(r'-?\d+', row)
		#print(nums)
		#print(all(num == 0 for num in nums))
		all_zero = all(int(num) == 0 for num in nums)
		#row = row.replace('{', '[').replace('}', ']')
	else:
		all_zero = pd.NA
	return all_zero

linkinfo['matrix_zero'] = linking_matrix.apply(lambda x: parse_matrix(x))


In [40]:
matrix_zero_df = linkinfo[['name_unoriented', 'matrix_zero']].drop_duplicates(keep='first')
#matrix_zero_df = matrix_zero_df.set_index(matrix_zero_df['name_unoriented'], drop=True)

matrix_zero_df.reset_index(drop=True)


,name_unoriented,matrix_zero
0,L2a1,False
1,L4a1,False
2,L5a1,True
3,L6a1,False
4,L6a2,False
...,...,...
1419,L11n455,False
1420,L11n456,False
1421,L11n457,False
1422,L11n458,False


In [41]:
"""
nonzero = linkinfo.loc[linkinfo['matrix_zero'] == False][['name_unoriented']].values.tolist()
zero = linkinfo.loc[linkinfo['matrix_zero']][['name_unoriented']].values.tolist()

links_with_nonzero_matrices = []
for item in nonzero:
	name = item[0]
	if name not in set(links_with_nonzero_matrices):
		links_with_nonzero_matrices.append(name)

links_with_zero_matrices = []
for item in zero:
	name = item[0]
	if name not in set(links_with_zero_matrices):
		links_with_zero_matrices.append(name)

links_with_nonzero_matrices = pd.Series(links_with_nonzero_matrices)
links_with_zero_matrices = pd.Series(links_with_zero_matrices)

#export list of links with nonzero linking matrices

links_with_nonzero_matrices.to_csv('results/nonzero_linking_matrix.csv')
links_with_nonzero_matrices.to_excel('results/nonzero_linking_matrix.xlsx')
"""


"\nnonzero = linkinfo.loc[linkinfo['matrix_zero'] == False][['name_unoriented']].values.tolist()\nzero = linkinfo.loc[linkinfo['matrix_zero']][['name_unoriented']].values.tolist()\n\nlinks_with_nonzero_matrices = []\nfor item in nonzero:\n\tname = item[0]\n\tif name not in set(links_with_nonzero_matrices):\n\t\tlinks_with_nonzero_matrices.append(name)\n\nlinks_with_zero_matrices = []\nfor item in zero:\n\tname = item[0]\n\tif name not in set(links_with_zero_matrices):\n\t\tlinks_with_zero_matrices.append(name)\n\nlinks_with_nonzero_matrices = pd.Series(links_with_nonzero_matrices)\nlinks_with_zero_matrices = pd.Series(links_with_zero_matrices)\n\n#export list of links with nonzero linking matrices\n\nlinks_with_nonzero_matrices.to_csv('results/nonzero_linking_matrix.csv')\nlinks_with_nonzero_matrices.to_excel('results/nonzero_linking_matrix.xlsx')\n"

In [42]:
determinant_is_zero = linkinfo['determinant'].apply(lambda row: row == 0)

#158 links whose determinant is zero
linkinfo['determinant_is_zero'] = determinant_is_zero
linkinfo['determinant_is_zero']


0       False
1       False
2       False
3       False
4       False
        ...  
4183     True
4184     True
4185     True
4186     True
4187     True
Name: determinant_is_zero, Length: 4188, dtype: bool

In [43]:
import math
#test if the determinant is a perfect square
determinant_is_square = linkinfo['determinant'].apply(lambda row: math.sqrt(row) % 1 == 0)
determinant_is_square

linkinfo['determinant_is_square'] = determinant_is_square


In [44]:
linkinfo.columns


Index(['name', 'name_anon', 'diagram', 'diagram_anon', 'name_unoriented',
       'category', 'knot_atlas', 'knot_atlas_anon', 'alternating',
       'orientation', 'name_rank', 'unoriented_name_rank', 'gauss_notation',
       'pd_notation_vector', 'pd_notation_math', 'crossing_number',
       'braid_index', 'braid_notation', 'braid_notation_old',
       'quasi_positive_braid', 'multivariable_alexander', 'conway_polynomial',
       'jones_polynomial', 'jones_polynomial_vector', 'homflypt_polynomial',
       'homflypt_polynomial_vector', 'kauffman_polynomial',
       'kauffman_polynomial_vector', 'khovanov_polynomial',
       'khovanov_polynomial_vector', 'unoriented', 'arc_notation',
       'linking_matrix', 'rolfsen_name', 'volume', 'components', 'dt_code',
       'arf_invariant', 'smooth_four_genus', 'topological_four_genus',
       'seifert_matrix', 'splitting_number', 'determinant', 'signature',
       'nullity', 'unlinking_number', 'weak_splitting_number', 'matrix_zero',
       'det

In [ ]:

#aggregate information needed to check whether link is ribbon (add more properties later)
ribbon_info = linkinfo[['name_unoriented', 'linking_matrix', 'components', 'determinant', 'matrix_zero', 'determinant_is_zero', 'smooth_four_genus']]
ribbon_info = ribbon_info.drop_duplicates(keep='first')

matrix_and_determinant_both_zero = ribbon_info['matrix_zero'] * ribbon_info['determinant_is_zero']

ribbon_info['matrix_and_determinant_both_zero'] = matrix_and_determinant_both_zero

matrix_and_determinant_both_zero.value_counts()

ribbon_info[matrix_and_determinant_both_zero].name_unoriented.tolist()


['L9n27',
 'L10n32',
 'L10n36',
 'L10n56',
 'L10n57',
 'L10n59',
 'L10n107',
 'L11n218',
 'L11n226',
 'L11n244',
 'L11n247',
 'L11n381',
 'L11n396',
 'L11n404',
 'L11n406',
 'L11n436']

In [47]:
#knots whose linking matrix are zero
ribbon_info[ribbon_info['matrix_zero']]


,name_unoriented,linking_matrix,determinant,matrix_zero,determinant_is_zero,smooth_four_genus,matrix_and_determinant_both_zero
4,L5a1,"{{0, 0}, {0, 0}}",8,True,False,1,False
11,L6a4,"{{0, 0, 0}, {0, 0, 0}, {0, 0, 0}}",16,True,False,1,False
20,L7a1,"{{0, 0}, {0, 0}}",24,True,False,1,False
23,L7a3,"{{0, 0}, {0, 0}}",16,True,False,1,False
24,L7a4,"{{0, 0}, {0, 0}}",16,True,False,0,False
...,...,...,...,...,...,...,...
3267,L11n404,"{{0, 0, 0}, {0, 0, 0}, {0, 0, 0}}",0,True,True,1,True
3270,L11n406,"{{0, 0, 0}, {0, 0, 0}, {0, 0, 0}}",0,True,True,0,True
3273,L11n408,"{{0, 0, 0}, {0, 0, 0}, {0, 0, 0}}",16,True,False,0,False
3274,L11n409,"{{0, 0, 0}, {0, 0, 0}, {0, 0, 0}}",80,True,False,1,False


In [48]:
#knots whose determinant (calculated by plugging -1 into the alexander polynomial) are zero
ribbon_info[ribbon_info['determinant_is_zero']]


,name_unoriented,linking_matrix,determinant,matrix_zero,determinant_is_zero,smooth_four_genus,matrix_and_determinant_both_zero
104,L8n6,"{{0, -2, 0}, {-2, 0, -2}, {0, -2, 0}}",0,False,True,1,False
105,L8n6,"{{0, 2, 0}, {2, 0, 2}, {0, 2, 0}}",0,False,True,2,False
106,L8n6,"{{0, -2, 0}, {-2, 0, 2}, {0, 2, 0}}",0,False,True,0,False
107,L8n6,"{{0, 2, 0}, {2, 0, -2}, {0, -2, 0}}",0,False,True,0,False
116,L8n8,"{{0, -1, 1, 0}, {-1, 0, 0, 1}, {1, 0, 0, -1}, ...",0,False,True,0,False
...,...,...,...,...,...,...,...
3517,L11n459,"{{0, 1, 1, 0}, {1, 0, 1, -1}, {1, 1, 0, -1}, {...",0,False,True,0,False
3518,L11n459,"{{0, -1, -1, 0}, {-1, 0, 1, -1}, {-1, 1, 0, -1...",0,False,True,0,False
3519,L11n459,"{{0, 1, -1, 0}, {1, 0, -1, 1}, {-1, -1, 0, -1}...",0,False,True,0,False
3520,L11n459,"{{0, -1, 1, 0}, {-1, 0, -1, -1}, {1, -1, 0, 1}...",0,False,True,0,False


In [49]:
ribbon_info[ribbon_info['matrix_zero'] & ribbon_info['matrix_zero']]


,name_unoriented,linking_matrix,determinant,matrix_zero,determinant_is_zero,smooth_four_genus,matrix_and_determinant_both_zero
4,L5a1,"{{0, 0}, {0, 0}}",8,True,False,1,False
11,L6a4,"{{0, 0, 0}, {0, 0, 0}, {0, 0, 0}}",16,True,False,1,False
20,L7a1,"{{0, 0}, {0, 0}}",24,True,False,1,False
23,L7a3,"{{0, 0}, {0, 0}}",16,True,False,1,False
24,L7a4,"{{0, 0}, {0, 0}}",16,True,False,0,False
...,...,...,...,...,...,...,...
3267,L11n404,"{{0, 0, 0}, {0, 0, 0}, {0, 0, 0}}",0,True,True,1,True
3270,L11n406,"{{0, 0, 0}, {0, 0, 0}, {0, 0, 0}}",0,True,True,0,True
3273,L11n408,"{{0, 0, 0}, {0, 0, 0}, {0, 0, 0}}",16,True,False,0,False
3274,L11n409,"{{0, 0, 0}, {0, 0, 0}, {0, 0, 0}}",80,True,False,1,False


In [50]:
#export the version of the linkinfo table with empty values removed and links with nonzero linking matrices identified in a separate column
ribbon_info.to_csv('results/cleaned_linkinfo.csv', index=False)


In [51]:
#takes roughly a minute to run
!sage "calculate_jones.sage"


sage script running
Traceback (most recent call last):
  File "/Users/seohyeonlee/Library/SageMath-10-6/lib/python3.12/site-packages/pandas/core/indexes/base.py", line 3812, in get_loc
    return self._engine.get_loc(casted_key)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "pandas/_libs/index.pyx", line 167, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 196, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 7088, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 7096, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'components'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/seohyeonlee/knotinfo/calculate_jones.sage.py", line 32, in <module>
    cleaned_linkinfo = cleaned_linkinfo.apply(factor_jones, axis=_sage_const_1 )
                       ^^^^^^^^^^^^^

In [52]:
sage_out = pd.read_csv('/Users/seohyeonlee/knotinfo/results/sage_output.csv')


FileNotFoundError: [Errno 2] No such file or directory: '/Users/seohyeonlee/knotinfo/results/sage_output.csv'

In [ ]:
out = sage_out[['name_unoriented', 'components', 'determinant', 'matrix_zero', 'jones_factorable', 'jones_determinant']].drop_duplicates()
out.to_csv('/Users/seohyeonlee/knotinfo/results/ribbon_shortlist.csv')
out.to_excel('/Users/seohyeonlee/knotinfo/results/ribbon_shortlist.xlsx')


In [ ]:
#16 links whose determinant is 0 and linking matrix has only zeroes
out[(out['determinant'] == 0) & out['matrix_zero'] ]


,name_unoriented,components,determinant,matrix_zero,jones_factorable,jones_determinant
346,L9n27,3,0,True,False,NaN
918,L10n32,2,0,True,True,-23.0
926,L10n36,2,0,True,True,-23.0
966,L10n56,2,0,True,True,17.0
968,L10n57,2,0,True,True,-15.0
972,L10n59,2,0,True,True,-23.0
1196,L10n107,4,0,True,False,NaN
3206,L11n218,2,0,True,True,-19.0
3222,L11n226,2,0,True,True,13.0
3258,L11n244,2,0,True,True,33.0


In [ ]:
#filtered links that satisfy 2 conditions (determinant 0, matrix only 0) and whose jones determinants are defined
out[(out['determinant'] == 0) & out['matrix_zero'] & out['jones_factorable']]


,name_unoriented,components,determinant,matrix_zero,jones_factorable,jones_determinant
918,L10n32,2,0,True,True,-23.0
926,L10n36,2,0,True,True,-23.0
966,L10n56,2,0,True,True,17.0
968,L10n57,2,0,True,True,-15.0
972,L10n59,2,0,True,True,-23.0
3206,L11n218,2,0,True,True,-19.0
3222,L11n226,2,0,True,True,13.0
3258,L11n244,2,0,True,True,33.0
3264,L11n247,2,0,True,True,-31.0
